In [5]:
import requests
import pandas as pd
import json
import numpy as np
from datetime import datetime, timezone
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
from sklearn.ensemble import IsolationForest
import os
from dotenv import load_dotenv
import time
import locale

In [6]:
# Connexion à Snowflake
load_dotenv(r'D:\DATA\2025-11-28_MSPR-1_2\Good-Air\.env')

ACOUNT_SNOWFLAKE = os.getenv('ACOUNT_SNOWFLAKE')
USER_SNOWFLAKE = os.getenv('USER_SNOWFLAKE')
PASSWORD_SNOWFLAKE = os.getenv('PASSWORD_SNOWFLAKE')
conn = snowflake.connector.connect(
    user=USER_SNOWFLAKE,
    password=PASSWORD_SNOWFLAKE,
    account=ACOUNT_SNOWFLAKE,  # ex: "abcd-xy12345.europe-west4.gcp"
    warehouse="COMPUTE_WH",
    database="GOOD_AIR",
    schema="SILVER"
)

In [ ]:
# Connexion à Snowflake
load_dotenv(r'D:\DATA\2025-11-28_MSPR-1_2\Good-Air\.env')

ACOUNT_SNOWFLAKE = os.getenv('ACOUNT_SNOWFLAKE')
USER_SNOWFLAKE = os.getenv('USER_SNOWFLAKE')
PASSWORD_SNOWFLAKE = os.getenv('PASSWORD_SNOWFLAKE')
conn = snowflake.connector.connect(
    user=USER_SNOWFLAKE,
    password=PASSWORD_SNOWFLAKE,
    account=ACOUNT_SNOWFLAKE,  # ex: "abcd-xy12345.europe-west4.gcp"
    warehouse="COMPUTE_WH",
    database="GOOD_AIR",
    schema="SILVER"
)

def run_anomaly_detection():
    try:
        # --- ETAPE 1 : RECUPERATION (READ) ---
        print("Récupération des données...")
        # On ne prend que les colonnes utiles pour le ML (ex: valeurs numériques)
        query = "SELECT RECORD_ID, AQI FROM FACT_AIR_QUALITY_RECORDS"
        df = pd.read_sql(query, conn)

        # --- ETAPE 2 : MACHINE LEARNING (PROCESS) ---
        print("Analyse des outliers en cours...")
        
        # Initialisation du modèle 
        # contamination=0.01 signifie qu'on s'attend à environ 1% d'anomalies
        model = IsolationForest(contamination=0.01, random_state=42)
        
        # Entraînement et prédiction sur la valeur à surveiller
        # Reshape est nécessaire si on a une seule feature
        df['anomaly_score'] = model.fit_predict(df[['AQI']])
        
        # Le modèle retourne -1 pour une anomalie et 1 pour normal.
        # On convertit cela en booléen ou texte pour Snowflake
        df['IS_OUTLIER'] = df['anomaly_score'].apply(lambda x: True if x == -1 else False)

        # On filtre pour ne garder que les outliers à renvoyer (ou tout le monde, selon votre choix)
        df_outliers = df[df['IS_OUTLIER'] == True][['RECORD_ID', 'IS_OUTLIER']]

        # Ajoute la date et l'heure actuelles à chaque ligne
        df_outliers['DETECTED_AT'] = pd.Timestamp.now()
        
        # --- ETAPE 3 : INJECTION (WRITE) ---
        if not df_outliers.empty:
            print(f"Injection de {len(df_outliers)} anomalies détectées dans Snowflake...")
            
            # Utilisation de write_pandas pour la performance (bien plus rapide que INSERT)
            # On écrit dans une table dédiée aux alertes
            success, n_chunks, n_rows, _ = write_pandas(
                conn, 
                df_outliers, 
                table_name='ANOMALY_ALERTS',
            )
            print(f"Succès : {n_rows} lignes insérées.")
        else:
            print("Aucune anomalie détectée.")
        
    finally:
        conn.close()

if __name__ == "__main__":
    run_anomaly_detection()

Récupération des données...
Analyse des outliers en cours...
   CITY_ID                         RECORD_ID  AQI  IS_OUTLIER
14   CT019  10a7ac561d1c672efc8a6583dbf0342a  150        True


C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_21900\2807236097.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
